In [21]:

# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import attrgetter
#pip install pmdarima
from pmdarima.arima import auto_arima
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
#pip install pystan
#conda install -c conda-forge fbprophet


In [22]:
data = pd.read_csv("Hack31.csv")
data.dtypes

date           object
production    float64
id              int64
dtype: object

In [23]:
# Format Date to datetime
data['date'] = pd.to_datetime(data['date'])

# Format Order_Demand to numeric
#data['production'] = data['production'].str.replace('(','-')
#data['production'] = data['production'].str.replace(')','')
#data['production'] = pd.to_numeric(data['production'])


# Sort data by period
data = data.sort_values('date').reset_index().drop('index',axis=1)

In [24]:
print("Data Types")
print(data.dtypes)
print("\t")

print(data.head())
print("\t")

print("Data Dimension",data.shape)
print("\t")

print("The number of practices is",len(data['id'].value_counts().index))
print("Period range is from",data['date'].min(),"to", data['date'].max())
print("Production is from",data['production'].min(),"to", data['production'].max())
print("\t")

print('Number of missing values by column',[sum(data[i].isnull()) for i in data.columns])
print('All missing values are in Date column.')

Data Types
date          datetime64[ns]
production           float64
id                     int64
dtype: object
	
        date  production   id
0 2010-01-15   176922.69  121
1 2010-01-15   260611.62   36
2 2010-01-15    75627.95    9
3 2010-01-15   672892.39   77
4 2010-01-15   204319.21  101
	
Data Dimension (27170, 3)
	
The number of practices is 284
Period range is from 2010-01-15 00:00:00 to 2020-12-15 00:00:00
Production is from -159292.21 to 1230909.86
	
Number of missing values by column [0, 0, 0]
All missing values are in Date column.


In [25]:
# Check percentage of negative values vs postive values
# Extract negative values and aggregate by practices
negative = data.loc[data['production'] < 0]
negative_pv = pd.pivot_table(negative, values='production',index=['id'], aggfunc=np.sum
                            ).rename(columns={'production':'Total_Neg'})

# Extract positve values and aggregate by product
positive = data.loc[data['production'] > 0]
positive_pv = pd.pivot_table(positive, values='production',index=['id'], aggfunc=np.sum)

# Add a column with corresponding total positive value by product and calculate percentage, sort = desc
negative_pv['Total_Pos'] = positive_pv.loc[positive_pv.index.isin(negative_pv.index),]
negative_pv['Percentage'] = abs(negative_pv['Total_Neg'])*100/negative_pv['Total_Pos']
negative_pv = negative_pv.sort_values('Percentage', ascending = False)

print("Percentage ranges from",negative_pv['Percentage'].min(),"to",negative_pv['Percentage'].max())

Percentage ranges from 2.3570836489817398e-05 to 0.589993353561256


In [15]:
# Examine production of several practices
print("Production of practice 250")
print(data.loc[data['id'] == 250].head(10))
print("\t")
print("production of practice 77")
print(data.loc[data['id'] == 77].head(10))

Production of practice 250
           date  production   id
77   2010-01-15   652002.20  250
148  2010-02-15   729654.75  250
265  2010-03-15   716543.92  250
398  2010-04-15   699941.31  250
523  2010-05-15   763221.23  250
590  2010-06-15   811092.81  250
709  2010-07-15   757147.62  250
845  2010-08-15   725968.49  250
1019 2010-09-15   793400.90  250
1106 2010-10-15   675761.31  250
	
production of practice 77
           date  production  id
3    2010-01-15   672892.39  77
229  2010-02-15   668600.79  77
337  2010-03-15   770422.95  77
442  2010-04-15   650435.10  77
489  2010-05-15   690726.56  77
617  2010-06-15   807983.70  77
736  2010-07-15   725549.20  77
922  2010-08-15   680681.63  77
1001 2010-09-15   660119.01  77
1079 2010-10-15   692352.14  77


In [27]:

# This concerns Date values, thus observations with missing valuse in Date column need to be removed first.
# Observations with missing values are extracted into a new table. Investigation into missing values will be 
# performed later.
data_null = data.loc[data['date'].isnull() == True,]
data = data.dropna(how='any',axis=0)
[sum(data[i].isnull()) for i in data.columns]

data['abs_production'] = abs(data['production'])

# Remove all rows that have the same products, dates, and absolute values of demand
data_nodup = data.drop_duplicates(subset = ['id','date','abs_production'], keep = False)

In [28]:
duplicates = data.iloc[~data.index.isin(data_nodup.index)]
#duplicates = pd.pivot_table(duplicates, values='abs_production',index=['id','date'],aggfunc=np.sum).reset_index()
#duplicates = duplicates.loc[duplicates['abs_production']>0]

In [29]:
# Remove all rows of duplicates in the original dataset, and drop Abs_production column
# Remove all negative values
#data = data.drop_duplicates(subset = ['id','date','abs_production'], keep = False)
data = data.drop('abs_production',axis=1)
data = data.loc[data['production']>0]

# Add duplicates table back to data
#data = pd.concat([data,duplicates], ignore_index = True)

# Check to see if any negative values remain
data.loc[data['production'] <0] # no rows

,date,production,id


In [30]:
data['date'] = data['date'].dt.to_period('M')
data = data.rename(columns = {"date": 'Period'})
data = data.groupby(['id','Period'])['production'].sum().reset_index().sort_values('Period'
            ).reset_index().drop('index',axis=1)
data.head()

,id,Period,production
0,166,2010-01,161550.34
1,100,2010-01,180204.95
2,167,2010-01,531647.50
3,207,2010-01,180808.61
4,165,2010-01,111058.49


In [31]:
data = pd.pivot_table(data, values = 'production', index = 'Period', columns = 'id',aggfunc=np.sum
                     ).reset_index().rename_axis("", axis="columns")

#Fill in missing values with 0. Months with missing values are implied to have zero demands.
data = data.fillna(0)
data = data.set_index('Period')
data.head()

,1,2,3,4,5,6,7,8,9,10,...,275,276,277,278,279,280,281,282,283,284
Period,,,,,,,,,,,,,,,,,,,,,
2010-01,0.0,0.0,132294.60,0.0,0.0,0.0,0.0,631655.18,75627.95,0.0,...,132294.60,0.0,195265.67,0.0,0.0,0.0,23500.30,131389.86,0.0,116898.96
2010-02,0.0,0.0,120940.38,0.0,0.0,0.0,0.0,573685.25,74750.23,0.0,...,120940.38,0.0,137335.58,0.0,0.0,0.0,22732.13,114578.90,0.0,143485.35
2010-03,0.0,0.0,129764.84,0.0,0.0,0.0,0.0,708231.05,86416.95,0.0,...,129764.84,0.0,225895.38,0.0,0.0,0.0,31182.41,121786.56,0.0,128807.91
2010-04,0.0,0.0,134429.64,0.0,0.0,0.0,0.0,614472.78,70451.64,0.0,...,134429.64,0.0,213497.07,0.0,0.0,0.0,29936.94,124297.02,0.0,105935.75
2010-05,0.0,0.0,176593.35,0.0,0.0,0.0,0.0,635609.52,78019.03,0.0,...,176593.35,0.0,208615.44,0.0,0.0,0.0,14637.22,139991.54,0.0,121439.16


In [33]:
def train_test(data):
    myList = data.tolist()
    i = myList.index(next(filter(lambda x: x!=0, myList)))
    data = data.iloc[i:,]
    train = data[:int(0.7*(len(data)))]
    test = data[int(0.7*len(data)):]
    return train, test, data

# create data for forecasting
start = data.index.tolist()[-1]
fcastperiods = 6  # forecast periods is subject to change by forecast users
full_period = [start + x for x in range(0,fcastperiods)]


In [34]:
#%%capture 

# build function to run model for all columns
Arima = ['Arima']
ArimaFcastPerf = pd.DataFrame({'Models': Arima})
ArimaData = pd.DataFrame({'Period': full_period, 'Model': 'Arima'})

def arimafcast(data):
    for i in data.columns:
        try:
            train, test, full = train_test(data[i])
            
            # Test model
            model_pred = auto_arima(train, start_p=0, start_q=0, max_p=3, max_q=3,
                                    m=1, seasonal=False, trace=True, 
                                    error_action='ignore', suppress_warnings=True)
            model_pred.fit(train)
            pred = np.round(model_pred.predict(n_periods=len(test)))
            ArimaFcastPerf[i] = sqrt(mean_squared_error(test,pred))
            
            # Forecast model
            model_fc = auto_arima(full, start_p=0, start_q=0, max_p=3, max_q=3,
                                    m=1, seasonal=False, trace=True, 
                                    error_action='ignore', suppress_warnings=True)
            model_fc.fit(full)
            forecast = np.round(model_fc.predict(n_periods=fcastperiods+2))
            ArimaData[i] = forecast[-fcastperiods:]
        except:
            ArimaFcastPerf[i] = np.nan
            ArimaData[i] = np.nan
    return ArimaFcastPerf, ArimaData
arimafcast(data)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=712.854, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=707.339, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=706.508, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=711.387, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=708.130, Time=0.03 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=708.680, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.14 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=706.383, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=708.322, Time=0.03 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=708.524, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=706.658, Time=0.01 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=710.313, Time=0.05 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 0.396 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=1068.973, Time=0.01

 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2946.524, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2965.606, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2947.968, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=2947.961, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=2946.347, Time=0.10 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=2948.327, Time=0.17 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=2948.531, Time=0.21 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=2949.790, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2949.714, Time=0.05 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=2949.044, Time=0.25 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=2944.656, Time=0.10 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=2946.194, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2946.290, Time=0.04 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=2946.639, Time=0.17 sec
 ARIMA(1,1,3)(0,0,0)[0]             : AIC=2946.820, Time=0.18 sec
 ARIMA(0,1

 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.30 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.23 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=2057.944, Time=0.13 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=2054.747, Time=0.21 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=2053.365, Time=0.14 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.32 sec
 ARIMA(2,1,3)(0,0,0)[0]             : AIC=2055.066, Time=0.15 sec

Best model:  ARIMA(2,1,3)(0,0,0)[0] intercept
Total fit time: 2.183 seconds
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=3378.948, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=3069.002, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=3297.917, Time=0.03 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=3046.794, Time=0.03 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=3035.179, Time=0.02 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=3035.810, Time=0.13 sec
 ARIMA(2,0,1)(0,0,0)[0]             :

 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=3041.633, Time=0.29 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=3042.856, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=3041.725, Time=0.07 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=3042.011, Time=0.21 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=3041.028, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=3044.462, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=3040.162, Time=0.06 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=3053.366, Time=0.01 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=3039.735, Time=0.03 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=3041.724, Time=0.04 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=3043.725, Time=0.09 sec

Best model:  ARIMA(2,0,0)(0,0,0)[0] intercept
Total fit time: 2.540 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2152.461, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2141.491, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0

 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3052.708, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3049.245, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3050.191, Time=0.09 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=3055.155, Time=0.08 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=3055.070, Time=0.12 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=3046.250, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=3047.104, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=3048.145, Time=0.10 sec
 ARIMA(0,1,3)(0,0,0)[0]             : AIC=3052.210, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=3044.946, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3071.233, Time=0.02 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=3046.689, Time=0.05 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=3061.340, Time=0.03 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=3047.755, Time=0.10 sec

Best model:  ARIMA(1,1,1)(0,0,0)[0]          
Total fit time: 0.993 seconds

...

 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=1704.237, Time=0.05 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=1711.290, Time=0.08 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.24 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=1702.036, Time=0.03 sec

Best model:  ARIMA(2,1,1)(0,0,0)[0] intercept
Total fit time: 0.946 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2525.606, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2521.394, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2511.572, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2523.611, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2503.126, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2502.532, Time=0.07 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=2513.352, Time=0.03 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=2504.486, Time=0.09 sec
 ARIMA(2,1,2)(0,0,0)[0] intercep

 ARIMA(0,1,2)(0,0,0)[0]             : AIC=3180.899, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=3175.384, Time=0.08 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=3187.077, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3190.068, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=3177.342, Time=0.07 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=3189.094, Time=0.03 sec

Best model:  ARIMA(1,1,1)(0,0,0)[0]          
Total fit time: 2.579 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=755.693, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=754.897, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=754.916, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=753.715, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=756.752, Time=0.03 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.083 seconds
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : A

 ARIMA(3,0,1)(0,0,0)[0]             : AIC=1337.272, Time=0.08 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=1335.718, Time=0.08 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=1339.077, Time=0.03 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=1336.833, Time=0.09 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=1334.917, Time=0.05 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=1431.664, Time=0.02 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=1336.823, Time=0.09 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=1430.208, Time=0.02 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=1338.578, Time=0.13 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=1329.885, Time=0.03 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=1328.265, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=1326.472, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=1333.602, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=1328.080, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=1326.280, Time=0.01 sec
 ARIMA(2,0

 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=2138.546, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=2129.221, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2130.774, Time=0.03 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=2130.841, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=2155.263, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=2133.636, Time=0.03 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 0.506 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3107.358, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3083.094, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3067.244, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3105.372, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3067.391, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3066.922, Time=0.04 sec
 ARIMA(1,1,2)(0,0,0)[0] int

(  Models             1             2             3             4  \
 0  Arima  21214.131603  33914.147394  96794.708728  42253.552282   
 
               5              6              7              8            9  \
 0  33732.983868  123459.123622  252876.311551  145388.470063  40174.28531   
 
    ...           275          276            277           278           279  \
 0  ...  96794.708728  82463.44159  109823.107894  58970.586838  59954.704927   
 
             280           281           282            283           284  
 0  38329.236895  49202.254692  33741.132614  159860.430175  58615.119372  
 
 [1 rows x 285 columns],
     Period  Model        1        2         3        4         5         6  \
 0  2020-12  Arima  62342.0  59792.0  222952.0  82803.0  122734.0  438114.0   
 1  2021-01  Arima  62342.0  59938.0  221481.0  82803.0  123411.0  431745.0   
 2  2021-02  Arima  62342.0  59998.0  220712.0  82803.0  119653.0  427558.0   
 3  2021-03  Arima  62342.0  60015.0  22031

In [ ]:
%%capture 

 #build function to run model for all columns
SArima = ['SArima']
SArimaFcastPerf = pd.DataFrame({'Models': SArima})
SArimaData = pd.DataFrame({'Period': full_period, 'Model': 'SArima'})

def sarimafcast(data):
    for i in data.columns:
        try:
            train, test, full = train_test(data[i])
            
            # Test model
            model_pred = auto_arima(train, start_p=0, start_q=0, max_p=3, max_q=3,
                                  m=12, trace=True, 
                                  error_action='ignore', suppress_warnings=True)
            model_pred.fit(train)
            pred = np.round(model_pred.predict(n_periods=len(test)))
            SArimaFcastPerf[i] = sqrt(mean_squared_error(test,pred))
            
            # Forecast model
            model_fc = auto_arima(full, start_p=0, start_q=0, max_p=3, max_q=3,
                                  m=12, trace=True, 
                                  error_action='ignore', suppress_warnings=True)
            model_fc.fit(full)
            forecast = np.round(model_fc.predict(n_periods=fcastperiods+2))
            SArimaData[i] = forecast[-fcastperiods:]
        except:
            SArimaFcastPerf[i] = np.nan
            SArimaData[i] = np.nan
    return SArimaFcastPerf, SArimaData
sarimafcast(data)

In [ ]:
%%capture

# build function to run model for all columns
SES = ['SES']
SESFcastPerf = pd.DataFrame({'Models': SES})
SESData = pd.DataFrame({'Period': full_period, 'Model': 'SES'})

def sesfcast(data):
    for i in data.columns:
        try:
            train, test, full = train_test(data[i])
                
            # Test model
            model_pred = SimpleExpSmoothing(train).fit()
            pred = np.round(model_pred.forecast(len(test)))
            SESFcastPerf[i] = sqrt(mean_squared_error(test,pred))
                
            # Forecast model
            model_fc = SimpleExpSmoothing(full).fit()
            forecast = np.round(model_fc.forecast(fcastperiods+2))
            SESData[i] = forecast.tolist()[-fcastperiods:]            
        except:
            SESFcastPerf[i] = np.nan
            SESData[i] = np.nan
    return SESFcastPerf, SESData
sesfcast(data)

In [ ]:

#Combine all performance tables
FcastPerf = pd.concat([ArimaFcastPerf,SArimaFcastPerf,SESFcastPerf]).set_index('Models')
FcastPerf

In [ ]:
Model = pd.DataFrame()

for i in FcastPerf.columns:
    # Find the one with lowest RMSE. Choose the first one in case of more than 1 min values.
    Model[i] = list([FcastPerf.loc[FcastPerf[i] == FcastPerf[i].min()].index[0]])
Model

In [ ]:
model_dict = {}
model_list = ['Arima','SArima', 'SES']
for i in model_list:
    model_dict[i] = []

for i in Model.columns:
    for j in model_list:
        if any(Model[i] == j):
            model_dict[j].append(i)
model_dict
AllForecast = pd.concat([ArimaData,SArimaData, SESData])
FinalForecast = pd.DataFrame({'Period': full_period})

for i in model_list:
    for j in model_dict[i]:
        FinalForecast[j] = AllForecast.loc[AllForecast['Model'] == i][j]
FinalForecast = FinalForecast.set_index('Period')

In [ ]:
# Combine historical and forecast data. Insert Data_Type column for filter purpose and move it to front.
HistoricalData = data[sorted(data)]
FinalForecast = FinalForecast[sorted(FinalForecast)]
FinalData = pd.concat([HistoricalData,FinalForecast])
    
FinalData['Data Type'] = np.nan
FinalData['Data Type'].iloc[:-fcastperiods] = 'Historical Data'
FinalData['Data Type'].iloc[-fcastperiods:] = 'Forecast'
co_list = FinalData.columns.tolist() 
co_list.insert(0, co_list.pop(co_list.index('Data Type'))) 
FinalData = FinalData.reindex(columns= co_list)

In [ ]:

# Save final data
FinalData.to_csv("FinalDatahack.csv")
FinalData

In [ ]:
for i in FinalData.columns[1:]:
    ax = FinalData[i].iloc[:-fcastperiods].plot(figsize=(15, 5), color ='green')
    FinalData[i].iloc[-fcastperiods:].plot(ax=ax, color ='red')
    fig = ax.get_figure()
    plt.show(block=False)
    plt.close(fig)